# Summarize large text

Anton Antonov   
August 2025

----

## Introduction

This notebook illustrates how to specify a Large Language Model (LLM) graph for deriving comprehensive summaries of large texts. 
The LLM graph is based on different LLM- and non-LLM functions .
The Raku package ["LLM::Graph"](https://raku.land/zef:antononcube/LLM::Graph) is used, [AAp1].

Using the LLM graph is an alternative to the Literate programming based solutions shown in [AA1, AAn1].

----

## Setup

In [1]:
use LLM::Graph;
use Data::Importers;

use LLM::Tooling;

In [2]:
sink my $engine = 'neato';

In [3]:
#sink my $conf = llm-configuration('ChatGPT', model => 'gpt-5-mini')
sink my $conf = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', temperature => 0.55, max-tokens => 4096)

----

## LLM graph

Specify the LLM graph nodes:

In [4]:
sink my %rules =
TypeOfInput => sub ($_) {
        "Determine the input type of\n\n$_.\n\nThe result should be one of: 'Text', 'URL', 'FilePath', or 'Other'."  ~ 
        llm-prompt('NothingElse')('single string')
    },

IngestText =>  { eval-function => sub ($TypeOfInput, $_) { $TypeOfInput ~~ / URL | FilePath/ ?? data-import($_) !! $_} },

Title => { 
    eval-function => sub ($IngestText, $with-title = Whatever) { $with-title ~~ Str:D ?? $with-title !! llm-synthesize(llm-prompt("TitleSuggest")($IngestText, 'article')) },
},

Summary => sub ($IngestText) { llm-prompt("Summarize")() ~ "\n\n$IngestText" },

TopicsTable => sub ($IngestText) { llm-prompt("ThemeTableJSON")($IngestText, 'article', 20) },

ThinkingHats => sub ($IngestText) { llm-prompt("ThinkingHatsFeedback")($IngestText, 'HTML') },

MindMap => sub ($IngestText) { 
    [
        "Create a concise mermaid-js mind-map for the text:",
        $IngestText,
        "All node messages should be in quotes.",
        llm-prompt("NothingElse")("correct mermaid-js")
    ]
},

Report => { eval-function => 
    sub ($Title, $Summary, $TopicsTable, $MindMap, $ThinkingHats) { 
        [
            "# $Title",
            '### *LLM summary report*',
            '## Summary',
            $Summary,
            '## Topics',
            to-html(
                from-json($TopicsTable.subst(/ ^ '```json' | '```' $/):g),
                field-names => <theme content>,
                align => 'left'),
            "## Mind map",
            $MindMap,
            '## Thinking hats',
            $ThinkingHats.subst(/ ^ '```html' | '```' $/):g
        ].join("\n\n")
    } 
},

ExportAndOpen => {
    eval-function => sub ($Report) {
       spurt('./Report.md', $Report);
       shell "open ./Report.md" 
    },
    test-function => -> $export-and-open = True { $export-and-open ~~ Bool:D && $export-and-open || $export-and-open.Str.lc ∈ <true yes open> }
}
;

Make the graph:

In [5]:
my $gCombinedSummary = LLM::Graph.new(%rules, llm-evaluator => $conf, :async)

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

----

## Full computation

URL and text statistics:

In [6]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/Graph-neat-examples-in-Raku-Set-3-YouTube.txt';
my $txtFocus = data-import($url);

text-stats($txtFocus)

(chars => 5861 words => 1158 lines => 156)

Computation:

In [7]:
$gCombinedSummary.eval($url, with-title => '«Graph» neat examples, set 3')

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

**Remark:** Instead of deriving the title using an LLM, the title is specified as an argument.

Show the corresponding graph:

In [8]:
#% html
$gCombinedSummary.dot(node-width => 1.6, edge-color => 'DimGrey', edge-width => 0.8):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
 
 ThinkingHats_Cluster 
 
 
 
 TypeOfInput_Cluster 
 
 
 
 TopicsTable_Cluster 
 
 
 
 MindMap_Cluster 
 
 
 
 Summary_Cluster 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- IngestText -->
 
 IngestText 
 
 IngestText 
 
<!-- $_->IngestText -->
 
 $_->IngestText 
 
 
 
<!-- TypeOfInput -->
 
 TypeOfInput 
 
 TypeOfInput 
 
<!-- $_->TypeOfInput -->
 
 $_->TypeOfInput 
 
 
 
<!-- ExportAndOpen -->
 
 ExportAndOpen 
 
 ExportAndOpen 
 
<!-- MindMap -->
 
 MindMap 
 
 MindMap 
 
<!-- IngestText->MindMap -->
 
 IngestText->MindMap 
 
 
 
<!-- Summary -->
 
 Summary 
 
 Summary 
 
<!-- IngestText->Summary -->
 
 IngestText->Summary 
 
 
 
<!-- ThinkingHats -->
 
 ThinkingHats 
 
 ThinkingHats 
 
<!-- IngestText->ThinkingHats -->
 
 IngestText->ThinkingHats 
 
 
 
<!-- Title -->
 
 Title 
 
 Title 
 
<!-- IngestText->Title -->
 
 IngestText->Title 
 
 
 
<!-- TopicsTable -->
 
 TopicsTable 
 
 TopicsTable 
 
<!-- IngestText->TopicsTable -->
 
 IngestText->TopicsTable 
 
 
 
<!-- Report -->
 
 Report 
 
 Report 
 
<!-- MindMap->Report -->
 
 MindMap->Report 
 
 
 
<!-- Report->ExportAndOpen -->
 
 Report->ExportAndOpen 
 
 
 
<!-- Summary->Report -->
 
 Summary->Report 
 
 
 
<!-- ThinkingHats->Report -->
 
 ThinkingHats->Report 
 
 
 
<!-- Title->Report -->
 
 Title->Report 
 
 
 
<!-- TopicsTable->Report -->
 
 TopicsTable->Report 
 
 
 
<!-- TypeOfInput->IngestText -->
 
 TypeOfInput->IngestText

Final result:

In [9]:
#% markdown
$gCombinedSummary.nodes<Report><result>.subst(/'```html' | '```' $/):g

# Visualizing Nested and File System Graphs in Raku: Practical Examples Using the Graph and Puffs Packages with JavaScript Rendering

### *LLM summary report*

## Summary

The presentation titled "Graph Neat Examples in Raku" by Anton Antonov demonstrates the use of the Raku package "Graph" to create nested and file system graphs with visualizations via JavaScript libraries like D3.js and FreeGS. Nested graphs are generated by applying functions repeatedly to values, producing symbolic representations and directed or undirected graphs, while file system graphs visualize directory structures using data from packages like Puffs. The examples include practical applications such as visualizing Mathematica notebooks and Raku documentation files, showcasing graph plotting and manipulation techniques.

## Topics

<table border="1"><thead><tr><th>theme</th><th>content</th></tr></thead><tbody><tr><td align=left>Introduction and Overview</td><td align=left>Presentation title, date, speaker introduction, and overview of neat examples in Raku with graph visualizations.</td></tr><tr><td align=left>Graph Package and Visualization Tools</td><td align=left>Description of Raku graph package and visualization using JavaScript libraries d3js and GraphVis framework.</td></tr><tr><td align=left>Nested Graphs Basic Examples</td><td align=left>Explanation of nested graphs using function application, graph nested class, and symbolic representation.</td></tr><tr><td align=left>Nested Graphs with Two Functions</td><td align=left>Applying two functions to initial value and visualizing nested graph with JavaScript and SVG.</td></tr><tr><td align=left>Mod Graph Example</td><td align=left>Mod graph creation by squaring, adding one, taking mod, producing directed graph with vertices 0 to 9.</td></tr><tr><td align=left>Range Graph Example</td><td align=left>Range graph demonstrating nesting steps, directed vs undirected graph, and vertex in/out degree calculations.</td></tr><tr><td align=left>Visualization Challenges</td><td align=left>Discussion on complexity and visualization challenges of busy graphs using JavaScript rendering.</td></tr><tr><td align=left>File System Graphs with Puffs Package</td><td align=left>Using Puffs to graph Mathematica notebooks directory structure with circular embedding and file types.</td></tr><tr><td align=left>File System Graphs of Raku Documentation</td><td align=left>Graphing Raku doc files from GitHub repo, force display layout, and node interaction challenges.</td></tr><tr><td align=left>Conclusion</td><td align=left>Final remarks and demonstration of returning to original plot, closing the presentation.</td></tr></tbody></table>

## Mind map

```mermaid
mindmap
  root(( "Graph Neat Presentation" ))
    subgraph "Introduction"
      title("Title: Graph Neat")
      presenter("Presenter: Anton Antonov")
      date("Date: November 28th, 2024")
      description("Neat examples: concise code with compelling visual/textual output")
      package("Using ru package graph")
      visualization("Visualization: JavaScript libraries d3js, freGS, graphVis (SVG)")
    end
    subgraph "Nested Graphs Examples"
      basic("Basic nested graphs")
      basic --> nestedClass("Use graph nested class")
      nestedClass --> args("Args: function, initial value, number of applications")
      example1("Example: square function applied multiple times")
      example2("Example: two functions f and g applied")
      svgDisplay("SVG display using JavaScript and freGS")
      modGraph("Mod graph: vertex value transformed by (x² + 1) mod 10")
      rangeGraph("Range graph: demonstrate one-step and two-step nesting")
      rangeGraph --> properties("Directed/undirected graph, vertex in-degree/out-degree")
      visualizationBusy("Busy graph visualization with freGS")
    end
    subgraph "File System Graphs"
      packagePuffs("Using package Puffs")
      source1("Mathematica notebooks directory")
      process1("Split directories and files, create edges from root down")
      stats1("186 files and edges")
      circularEmbedding("Circular embedding visualization")
      showFilenames("Show file names (CSV, ZIP, NB)")
      source2("Raku doc files from GitHub clone")
      graph2("Graph with force display, squashed layout")
      nodeNames("Show node names (comp unit, hash Raku doc, etc.)")
      interaction("Nodes can be moved for better visibility")
    end
    subgraph "Conclusion"
      lastExample("Last neat example")
      thanks("Thank you")
    end
```

## Thinking hats

<table border="1" cellpadding="5" cellspacing="0">
<tr>
<th>Hat Name</th>
<th>Perspective</th>
<th>Feedback</th>
<th>New Ideas</th>
</tr>
<tr>
<td>White Hat</td>
<td>Information and Facts</td>
<td>The presentation clearly explains the use of the Raku package `Graph` and its visualization via JavaScript libraries such as D3.js and `freGS`. It covers nested graphs, mod graphs, range graphs, and file system graphs with concrete examples and code snippets. The explanation includes graph properties like vertex in-degree and out-degree, and the use of different embedding and rendering techniques. The factual content is accurate and well-structured, providing both conceptual and practical insights into graph construction and visualization in Raku.</td>
<td>Include performance metrics or benchmarks of graph rendering for large datasets, Add comparisons with alternative graph visualization tools in Raku or other languages, Provide links to official documentation or tutorials for the packages used.</td>
</tr>
<tr>
<td>Black Hat</td>
<td>Judgement and Caution</td>
<td>The presentation assumes familiarity with Raku and JavaScript, which may alienate beginners. The examples, especially the nested and mod graphs, may be too abstract or complex without more detailed explanations. The visualizations are described as "busy" and "not very nice," indicating potential usability issues. There is also a risk that relying on older frameworks like GraphViz might limit modern capabilities or performance. Handling large graphs (e.g., 186 files) could lead to cluttered visuals and slow rendering.</td>
<td>Provide more beginner-friendly explanations or tutorials, Simplify or break down complex examples into smaller parts, Explore modern or alternative graph visualization frameworks for better performance and clarity.</td>
</tr>
<tr>
<td>Gray Hat</td>
<td>Cynicism and Skepticism</td>
<td>This presentation risks being another "tech demo" that impresses with complexity but lacks practical impact. The use of multiple layers of abstraction (Raku, JavaScript, SVG, GraphViz) might be a convoluted way to solve problems that simpler tools could address. The heavy reliance on niche packages and old frameworks may reflect an agenda to promote Raku-specific tools rather than universally useful solutions. Also, the emphasis on "neat" code could mask the difficulty in maintaining or extending these graphs in real projects.</td>
<td>Question the necessity of the multi-layered tech stack, Investigate if the visualization approach is more about showcasing Raku than solving real-world problems, Consider streamlining the toolchain to focus on usability and maintainability.</td>
</tr>
<tr>
<td>Red Hat</td>
<td>Feelings and Emotion</td>
<td>The presentation feels enthusiastic and passionate about Raku and graph visualization, which is engaging. However, the complexity and technical jargon might overwhelm some listeners. The visual outputs, while "compelling," could cause frustration if they are cluttered or hard to interpret. The speaker’s tone is somewhat informal and hesitant at times ("probably," "should have"), which may reduce confidence in the material. Overall, the emotional tone conveys excitement but could benefit from more clarity and assurance.</td>
<td>Use more confident and clear language to build trust, Include more visuals or live demos to engage the audience emotionally, Add anecdotes or real-world use cases to connect emotionally with users.</td>
</tr>
<tr>
<td>Yellow Hat</td>
<td>Benefits and Creativity</td>
<td>This presentation offers a creative way to visualize complex graph structures using Raku, bridging coding and graphical representation effectively. The use of nested functions and modular graph construction is innovative and showcases Raku's expressive power. Visualizing file systems as graphs is a compelling idea that can aid in understanding directory structures and dependencies. The integration with JavaScript libraries opens opportunities for interactive and dynamic graph exploration.</td>
<td>Expand interactivity in visualizations (e.g., zoom, filter, highlight), Develop educational tools to teach graph theory using these examples, Create templates for common graph types to speed up usage.</td>
</tr>
<tr>
<td>Green Hat</td>
<td>Profitability and Potential</td>
<td>The idea has potential in niche markets such as software visualization, educational tools, and data analysis within Raku communities. There is room to monetize by offering enhanced visualization packages, consulting on graph-based data representation, or creating commercial plugins for popular IDEs. Given the growing interest in graph databases and network analysis, a polished and user-friendly Raku graph visualization toolkit could attract paying users or sponsors.</td>
<td>Package the tools as a commercial Raku visualization suite, Offer training workshops or webinars on graph visualization in Raku, Develop integrations with enterprise graph databases for added value.</td>
</tr>
<tr>
<td>Blue Hat</td>
<td>Usefulness and Opportunities</td>
<td>The presentation’s approach is useful for developers and researchers needing to visualize functional transformations, file systems, or complex relationships programmatically. Opportunities exist to extend the toolkit for real-time data visualization, debugging code flows, or mapping dependencies in large projects. The file system graph visualization could be expanded to monitor file changes or permissions, providing practical benefits beyond static images.</td>
<td>Add real-time or dynamic graph update capabilities, Integrate with version control systems to visualize commit histories, Develop plugins for IDEs to visualize code dependencies on the fly.</td>
</tr>
<tr>
<td>Purple Hat</td>
<td>Assessment and Direction</td>
<td>Summarizing the perspectives, the idea is technically solid and creatively promising but needs refinement for usability and clarity. The skepticism and caution highlight the need for simplification and modern tooling. Emotional feedback suggests enhancing confidence and engagement. The potential for profitability and usefulness is clear, especially if the toolkit is expanded with interactive and real-time features. Directionally, focusing on user experience improvements, clearer documentation, and exploring commercial applications would strengthen the project.</td>
<td>Focus on improving visualization clarity and interactivity, Develop beginner-friendly tutorials and documentation, Explore commercial and educational applications to broaden impact.</td>
</tr>
</table>

---

## Partial evaluation

Drop the results in `LLM::Graph` computed above:

In [10]:
$gCombinedSummary.drop-result

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

Here the are normalized nodes without results:

In [11]:
.say for |$gCombinedSummary.nodes

ThinkingHats => {eval-function => sub { }, input => [IngestText], spec-type => (Routine), test-function-input => [], wrapper => Routine::WrapHandle.new}
Title => {eval-function => sub { }, input => [IngestText], spec-type => (Callable), test-function-input => []}
TypeOfInput => {eval-function => sub { }, input => [$_], spec-type => (Routine), test-function-input => [], wrapper => Routine::WrapHandle.new}
Report => {eval-function => sub { }, input => [Title MindMap Summary TopicsTable ThinkingHats], spec-type => (Callable), test-function-input => []}
TopicsTable => {eval-function => sub { }, input => [IngestText], spec-type => (Routine), test-function-input => [], wrapper => Routine::WrapHandle.new}
MindMap => {eval-function => sub { }, input => [IngestText], spec-type => (Routine), test-function-input => [], wrapper => Routine::WrapHandle.new}
IngestText => {eval-function => sub { }, input => [TypeOfInput $_], spec-type => (Callable), test-function-input => []}
ExportAndOpen => {eval-f

Here all results are pre-assigned as arguments:

In [12]:
$gCombinedSummary.eval($url, 
with-title => '«Graph» neat examples, set 3', 
:export-and-open,
Summary => 'In brief',
TypeOfInput => 'Other',
IngestText => 'Ingest text',
ThinkingHats => 'Thinking hats',
TopicsTable => '["TopicsTable"]',
MindMap => 'Mind-map graph',
)

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

-----

## References

### Blog posts

[AA1] Anton Antonov,
["Parameterized Literate Programming"](https://rakuforprediction.wordpress.com/2025/06/21/parameterized-literate-programming/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Notebooks

[AAn1] Anton Antonov,
["LLM comprehensive summary template for large texts"](https://community.wolfram.com/groups/-/m/t/3448842),
(2025),
[Wolfram Community](https://community.wolfram.com).

### Packages

[AAp1] Anton Antonov, 
[LLM::Graph, Raku package](https://github.com/antononcube/Raku-LLM-Graph),
(2025),
[GitHub/antononcube](https://github.com/antononcube).